<a href="https://colab.research.google.com/github/NeoMorpheus99/Natural-Language-Processing/blob/main/Multi_class_textClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

In [3]:
import pandas as pd
import torch
import transformers
from transformers import BertTokenizer
import numpy as np
from transformers import BertModel
from tqdm import tqdm

In [4]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [5]:
df=pd.read_csv("bbc-text.csv")

In [6]:
df['category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [7]:
#check for NaN values and Missing Data
df.isna().sum()

category    0
text        0
dtype: int64

In [8]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [9]:
#ecoding the labels of the category
# from sklearn import preprocessing
# label_encoder=preprocessing.LabelEncoder()
# df["En_category"] = label_encoder.fit_transform(df['category'])

In [10]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [12]:
#print(label_encoder.classes_)

In [13]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 1e-05

In [14]:
#imprt tokenziser

In [15]:
from transformers import BertTokenizer

In [16]:
tokenizer = transformers.BertTokenizerFast.from_pretrained('bert-base-uncased')

In [17]:
labels = {
    'business':0,
    'entertainment':1,
    'sport':2,
    'tech':3,
    'politics':4
}

In [18]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,df):
        #extract our labels from the df
        self.labels = [labels[label] for label in df["category"]]
        #tokenize our texts to the format that BERT expects to get as input
        self.texts = [tokenizer(text, padding='max_length', max_length=512, truncation=True,return_tensors="pt") for text in df["text"]]
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    #fetch a batch of labels
    def get_batch_labels(self,indx):
        return np.array(self.labels[indx])
    # fetch a batch of texts
    def get_batch_texts(self,indx):
        return self.texts[indx]

    #get an item with the texts and the label
    def __getitem__(self,indx):
        batch_texts = self.get_batch_texts(indx)
        batch_y = self.get_batch_labels(indx)


        return batch_texts, batch_y

In [19]:
df_train, df_valid,df_test = np.split(df.sample(frac=1,random_state=42),[int(.8*len(df)), int(.9*len(df))])

In [20]:
class BertClassifier(torch.nn.Module):
    def __init__(self,dropout=0.5):
        super(BertClassifier,self).__init__()

        self.bert=BertModel.from_pretrained("bert-base-cased")
        self.dropout = torch.nn.Dropout(dropout)
        # bert output a vector of size 768
        self.lin = torch.nn.Linear(768,5)
        self.relu = torch.nn.ReLU()
    def forward(self,input_id,mask):
        # as output, the bert model give us first the embedding vector of all the tokens of the sequence
        # second we get the embedding vector of the CLS token.
        # fot a classification task it's enough to use this embedding for our classifier
        _,pooled_output = self.bert(input_ids= input_id,attention_mask = mask,return_dict = False)
        dropout_output = self.dropout(pooled_output)
        linear_output  = self.lin(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


In [21]:
# we are creating a standard pytorch training loop

def train(model, train_data, val_data, learning_rate, epochs=5):
    #creating a custom Dataset objects using the training and validation data
    train, val = Dataset(train_data), Dataset(val_data)
    #creating dataloaders
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):
                #print(f"the train input : {train_input}")
                #print(f"train label : {train_label}")

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)
    #             print(input_id.shape)

                # get the predictions
                output = model(input_id, mask)

                #the output is a vector of 5 values (categs)
    #             print(output)
    #             print("the output shape is" ,  output.shape)
    #             print(train_label)

                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
                # updating the Gradient Descent and Backpropagation operation
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            # now we evaluate on the validation data
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')

In [22]:
EPOCHS = 5
model = BertClassifier()
learning_rate = 1e-6
train(model, df_train, df_valid, learning_rate, EPOCHS)

  0%|          | 1/890 [00:17<4:12:35, 17.05s/it]


IndexError: ignored

In [ ]:
def evaluate(model,test_df):
    test = Dataset(test_df)
    test_dl = torch.utils.data.DataLoader(test,batch_size=2)

    cuda_available = torch.cuda.is_available()

    device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
    if cuda_available:
        model = model.cuda()

    total_acc = 0
    for test_input , test_label in tqdm(test_dl):
        test_label = test_label.to(device)
        mask = test_input["attention_mask"].to(device)
        input_id = test_input["input_ids"].squeeze(1).to(device)
        output = model(input_id,mask)

        acc = (output.argmax(dim=1) == test_label).sum().item()
        total_acc +=acc

    print(f"Test Accuracy : {total_acc / len(test_df): .3f}")


In [ ]:
evaluate(model,test_df=df_test)

In [ ]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
inverse_labels = {v:k for k,v in labels.items()}

In [ ]:
def predict(device,model,sentence):
    sentence = sentence
    sentence_input = tokenizer(sentence, padding='max_length', max_length=512, truncation=True,return_tensors="pt").to(device)
    input_id = sentence_input["input_ids"]
    mask = sentence_input["attention_mask"]
    output = model(input_id,mask)
    predicted_class_label = output.argmax(dim=1)
    predicted_class = inverse_labels[predicted_class_label.item()]
    print(f"The predicted class is : {predicted_class}")